In [1013]:
# PROJECT 1: Data scraping and cleaning
    


In [1014]:
import pandas as pd
import numpy as np
pdfs = pd.read_html('https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares')



In [1015]:
## Step 1: Scrape your competitor's data

In [1016]:
top50_flares = pdfs[0]
top50_flares = top50_flares.rename(columns = {'Unnamed: 0': 'rank', 'Unnamed: 1': 'flare_classification','Unnamed: 2':'date', 'Region':'flare_region', 'Start':'start_time','Maximum':'maximum_time','End':'end_time','Unnamed: 7':'movie'})
top50_flares.head()

,rank,flare_classification,date,flare_region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


In [1017]:
## Step 2: Tidy the top 50 solar flare data

In [1018]:
top50_flares = top50_flares.drop(['movie'],axis=1)

top50_flares['start_datetime'] = pd.to_datetime(top50_flares['date'].str.cat(top50_flares['start_time'],sep=' '))
top50_flares['max_datetime'] =  pd.to_datetime(top50_flares['date'].str.cat(top50_flares['maximum_time'],sep=' '))
top50_flares['end_datetime'] = pd.to_datetime(top50_flares['date'].str.cat(top50_flares['end_time'],sep=' '))

top50_flares = top50_flares.drop(['start_time'],axis=1)
top50_flares = top50_flares.drop(['maximum_time'],axis=1)
top50_flares = top50_flares.drop(['end_time'],axis=1)
top50_flares = top50_flares.drop(['date'],axis=1)

top50_flares.head()

,rank,flare_classification,flare_region,start_datetime,max_datetime,end_datetime
0,1,X28+,486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00


In [1019]:
## Step 3: Scrape the NASA data
### The following code uses BeautifulSoup to get the table from the 'pre' element. Then parses the values in each string and uses these values to create a dataframe.


In [1020]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html')
soup = BeautifulSoup(res.text,'lxml')
soup = soup.find("pre").text.splitlines()
df = pd.DataFrame(columns=['start_date','start_time',
                            'end_date','end_time','start_frequency',
                            'end_frequency','flare_location','flare_region',
                            'flare_classification','cme_date','cme_time','cme_angle',
                            'cme_width','cme_speed'])
for sp in soup:
    if (len(sp.split()) >= 15):
        sp1 = sp.split()
        df = df.append({'start_date':sp1[0],'start_time':sp1[1],
                            'end_date':sp1[2],'end_time':sp1[3],'start_frequency':sp1[4],
                            'end_frequency':sp1[5],'flare_location':sp1[6],'flare_region':sp1[7],
                            'flare_classification':sp1[8],'cme_date':sp1[9],'cme_time':sp1[10],'cme_angle':sp1[11],
                            'cme_width':sp1[12],'cme_speed':sp1[13]}, ignore_index=True)
df = df.drop([0], axis = 0)
df.head()
        


,start_date,start_time,end_date,end_time,start_frequency,end_frequency,flare_location,flare_region,flare_classification,cme_date,cme_time,cme_angle,cme_width,cme_speed
1,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312
2,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,Halo,360,878
3,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,Halo,360,464
4,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296
5,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712


In [1021]:
## Step 4:Tidy the NASA table
### This finds and filters missing values 

In [1022]:
df = df.replace(to_replace=['----------','---','--','----','-----','------','????','--/--','--:--'], value=np.nan)
df['cme_angle_halo'] = np.where(df['cme_angle']=='Halo',True,False)
df = df.replace(to_replace='Halo', value=np.nan)
df.head()



,start_date,start_time,end_date,end_time,start_frequency,end_frequency,flare_location,flare_region,flare_classification,cme_date,cme_time,cme_angle,cme_width,cme_speed,cme_angle_halo
1,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312,False
2,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,NaN,360,878,True
3,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,NaN,360,464,True
4,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296,False
5,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712,False


In [1023]:
df['cme_width_lower'] = pd.np.where(df.cme_width.str.contains('NaN'),False,
                                   pd.np.where(df.cme_width.str.contains('>'), True, False))
df['cme_width'] = df['cme_width'].str.replace('>','')
df.head()

,start_date,start_time,end_date,end_time,start_frequency,end_frequency,flare_location,flare_region,flare_classification,cme_date,cme_time,cme_angle,cme_width,cme_speed,cme_angle_halo,cme_width_lower
1,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312,False,False
2,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,NaN,360,878,True,False
3,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,NaN,360,464,True,False
4,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296,False,False
5,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712,False,False


In [1024]:
df['start_datetime'] = pd.to_datetime(df['start_date'].str.cat(df['start_time'],sep=' '),errors='coerce')
df['end_datetime'] = pd.to_datetime(df['start_date'].str.cat(df['end_time'],sep=' '),errors='coerce')
df['cme_datetime'] = pd.to_datetime(df['start_date'].str.cat(df['cme_time'],sep=' '), errors='coerce')


In [1025]:
df = df.drop(['start_time'], axis=1)
df = df.drop(['end_time'], axis=1)
df = df.drop(['end_date'], axis=1)
df = df.drop(['start_date'],axis=1)
df.head()

,start_frequency,end_frequency,flare_location,flare_region,flare_classification,cme_date,cme_time,cme_angle,cme_width,cme_speed,cme_angle_halo,cme_width_lower,start_datetime,end_datetime,cme_datetime
1,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312,False,False,1997-04-01 14:00:00,1997-04-01 14:15:00,1997-04-01 15:18:00
2,11000,1000,S28E19,8027,C6.8,04/07,14:27,NaN,360,878,True,False,1997-04-07 14:30:00,1997-04-07 17:30:00,1997-04-07 14:27:00
3,12000,80,N21W08,8038,C1.3,05/12,05:30,NaN,360,464,True,False,1997-05-12 05:15:00,1997-05-12 16:00:00,1997-05-12 05:30:00
4,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296,False,False,1997-05-21 20:20:00,1997-05-21 22:00:00,1997-05-21 21:00:00
5,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712,False,False,1997-09-23 21:53:00,1997-09-23 22:16:00,1997-09-23 22:02:00


In [1026]:
df['start_frequency'] = pd.to_numeric(df['start_frequency'],downcast='integer')
df['end_frequency'] = pd.to_numeric(df['end_frequency'],downcast='integer')
df['cme_angle'] = pd.to_numeric(df['cme_angle'],downcast='integer')
df['cme_width'] = pd.to_numeric(df['cme_width'],downcast='integer')
df['cme_speed'] = pd.to_numeric(df['cme_speed'],downcast='integer')
df['flare_location'] = pd.to_string(df['flare_location'])
df['flare_classification'] = pd.to_string(df['flare_classification'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 518 entries, 1 to 518
Data columns (total 15 columns):
start_frequency         514 non-null float64
end_frequency           514 non-null object
flare_location          510 non-null object
flare_region            424 non-null object
flare_classification    405 non-null object
cme_date                496 non-null object
cme_time                495 non-null object
cme_angle               211 non-null object
cme_width               496 non-null object
cme_speed               496 non-null object
cme_angle_halo          518 non-null bool
cme_width_lower         518 non-null bool
start_datetime          518 non-null datetime64[ns]
end_datetime            504 non-null datetime64[ns]
cme_datetime            495 non-null datetime64[ns]
dtypes: bool(2), datetime64[ns](3), float64(1), object(9)
memory usage: 57.7+ KB
